# Los Alamos Authentication Classification

The following solution is a meta analysis of classifiers available in __[`scikit-learn`](http://scikit-learn.org/stable/index.html)__ partially based on the following examples:
 - http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py
 - http://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html#sphx-glr-auto-examples-model-selection-plot-randomized-search-py
***
To avoid memory limitations within Python, native commands are used via the `subprocess.Popen` class to download and decompress the archived auth.txt.gz file if necessary and find the population size and probability of successful authentication for sampling.  Through manual examination of the dataset the high success rate of authentication became apparent so with prior knowledge of the statistically correct technique to determine sample size (formula __[here](https://www.surveysystem.com/sample-size-formula.htm)__) given a confidence level and interval a relatively small number of elements was needed.  The original number of records exceed one billion but given the `p * (1 - p)` terms in the formula the samples required for a 99.5% confidence level with a $\pm$0.5% interval are 3202 elements, only about 0.0003% of the total data and much more practical to train the classifiers with.

Given the scope of hyperparameters for most of the classifiers an exhaustive grid search was not within reason so a randomized search over predefined ranges of paramters was chosen for training and validation.  Through development of this notebook two of the classifiers, `SCV` and `GaussianProcessClassifier`, were significantly more resource intensive without being more accurate so their options are commented out and the results not included in the analysis.

The timestamps included in the output provide an estimate for how long the commands run on the following system:
 - i7-3632QM CPU at 2.20 GHz
 - 16 GB of memory
 - 512 GB solid state drive
 - 200 MB internet connection over 5 GHz wireless router
<br>

Please review the documentation and timestamps before running the notebook (especially the note pertaining to downloading and decompressing the auth.txt.gz file).
***
 - System Requirements
    - Operating System = Linux based with "gunzip", "wc", "grep", and "shuf" commands available
    - Python version  = 2.7.5
    - Internet connection with access to https://csr.lanl.gov/data/cyber1/auth.txt.gz if archive not available locally
    - ~80 GB free space on partition of local directory notebook is executed in (if auth.txt.gz and auth_decompressed.txt files are not available locally)
 - Version numbers of included third party packages:
    - jupyter - 1.0.0
    - numpy - 1.14.5
    - pandas - 0.23.3
    - requests - 2.6.0
    - scikit-learn - 0.20.0
    - scipy - 1.1.0

All the needed imports

In [1]:
import datetime
import io
import math
import os
import subprocess
import warnings

import numpy
import pandas
import requests
import scipy.stats
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import ConstantKernel
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process.kernels import RationalQuadratic
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

Download the auth.txt.gz data (description available __[here](https://csr.lanl.gov/data/cyber1/)__) and decompress if either is necessary.  Chunking example to accommodate memory limitations of Python found __[here](https://stackoverflow.com/questions/16694907/how-to-download-large-file-in-python-with-requests-py)__.
 - This notebook was run without the auth.txt.gz or auth_decompressed.txt present in the local directory, note that over three hours were spent downloading and decompressing the file which can be avoided if they are available before running.

In [2]:
url = "https://csr.lanl.gov/data/cyber1/auth.txt.gz"
file_name = url.split("/")[-1]
decompressed_file_name = "auth_decompressed.txt"

print "[%s] Downloading %s if not available" \
    % (datetime.datetime.now(), url)

# Download the compressed authorization file if not available in the local
# directory
if not (os.path.exists(file_name) and os.path.exists(decompressed_file_name)):
    authorization_file = requests.get(url, stream=True)
    with open(file_name, "wb") as destination:
        for chunk in authorization_file.iter_content(chunk_size=1024): 
            if chunk:
                destination.write(chunk)

print "[%s] Decompressing %s if not available" \
    % (datetime.datetime.now(), file_name)

# Decompress the authorization file if not available in the local directory
if not os.path.exists(decompressed_file_name):
    with open("auth_decompressed.txt", "wb") as decompressed_file:
        subprocess.call(["gunzip", "-c", "auth.txt.gz"], stdout=decompressed_file)

[2018-10-14 18:22:23.449408] Downloading https://csr.lanl.gov/data/cyber1/auth.txt.gz if not available
[2018-10-14 21:56:31.376566] Decompressing auth.txt.gz if not available


Find the needed sample size given the population, probability of successful authentication, and chosen confidence level and interval.  Formula based on documentation found __[here](https://www.surveysystem.com/sample-size-formula.htm)__.

In [3]:
print "[%s] Calculating sample size" % datetime.datetime.now()

# Get the number of records as the population of data
p = subprocess.Popen(["wc", "-l", decompressed_file_name],
                     stdout=subprocess.PIPE)
row_count = int(p.stdout.read().split()[0])

# Z score for 99.5% confidence level
z_score = scipy.stats.norm.ppf(0.995)

# Calculate the probability of successful authentication
p = subprocess.Popen(["grep", "-c", ",Success",
                     decompressed_file_name], stdout=subprocess.PIPE)
probability_success = (int(p.stdout.read()) + 0.0) / row_count

# Confidence interval of 0.5%
confidence_interval = 0.005

# Initial sample size for infinite population
sample_size = z_score ** 2 * probability_success * (1
        - probability_success) / confidence_interval ** 2

# Correction for finite population
sample_size = int(math.ceil(sample_size / (1 + (sample_size - 1)
                  / row_count)))

sample_reduction = "{0:.6f}".format(100 * (1 - (sample_size + 0.0)
                                    / row_count))

print "Population = %s" % row_count
print "Z score = %s" % z_score
print "Probability of successful authorization = %s" \
    % probability_success
print "Required sample size = %s" % sample_size
print "Reduction of original data to sample = %s%%" % sample_reduction

[2018-10-14 22:05:43.929428] Calculating sample size
Population = 1051430459
Z score = 2.5758293035489004
Probability of successful authorization = 0.987787772467
Required sample size = 3202
Reduction of original data to sample = 99.999695%


Select elements using a uniform random distribution to collect the sample.

In [4]:
print "[%s] Collecting sample" % datetime.datetime.now()

# Randomly select rows of decompressed file to create sample
p = subprocess.Popen(["shuf", "-n", str(sample_size),
                     decompressed_file_name], stdout=subprocess.PIPE)

# Convert to two dimensional numpy array
sample = numpy.genfromtxt(io.BytesIO(p.stdout.read()), dtype="|U100",
                          delimiter=",", autostrip=True)

[2018-10-14 22:10:39.014814] Collecting sample


Create the training, validation, and verification sets based on the sample.  75% of the sample is used for a combination of training and validation data, the exact percentages for each are determined at runtime by the randomized parameter search for the classifiers listed below.  The remaining 25% is used for the verification set after the optimal parameters have been found through searching.  Definitions for the training, validation, and verification sets are available __[here](https://machinelearningmastery.com/difference-test-validation-datasets/)__.

In [5]:
print "[%s] Creating training/validation and verification sets" \
    % datetime.datetime.now()

# Create a mapping to assign integer values to input strings
mapping_keys = set(numpy.ndarray.tolist(numpy.ndarray.flatten(sample)))
mapping = {k: v for k,v in zip(mapping_keys, range(len(mapping_keys)))}

# Map all the string values to integers for classifiers
for i in range(sample.shape[0]):
    for j in range(sample.shape[1]):
        sample[i, j] = mapping[sample[i, j]]

# Convert integer unicode strings to integer values
sample = sample.astype(int, copy=False)

# Allocate which rows of the sample are for training/validation and
# which are for verification
train_validate_rows = range(0, int(0.75 * sample_size))
verification_rows = range(int(0.75 * sample_size), sample_size)

# Split sample into train/validate and verify sets.
x_train_validate = sample[train_validate_rows, :-1]
y_train_validate = sample[train_validate_rows, -1]
x_verify = sample[verification_rows, :-1]
y_verify = sample[verification_rows, -1]

[2018-10-14 22:14:57.681261] Creating training/validation and verification sets


Set up the classifiers with ranges of options for the various parameters of each.

In [6]:
print "[%s] Setting up classifier parameter options" \
    % datetime.datetime.now()

# Define parameters and associated ranges for each classifier
clf_opts = {
    "Nearest Neighbors": (KNeighborsClassifier(), {
        "n_neighbors": range(1, 16),
        "weights": ["uniform", "distance"],
        "algorithm": ["ball_tree", "kd_tree", "brute"],
        "leaf_size": range(10, 51, 10),
        "p": range(1, 3),
        }),
# Inclusion of SVC and Gaussian Process classifiers greatly increase
# the overall training duration without a comparable increase in
# validation or verification accuracy so they are only included here
# for reference
#     "SVM": (SVC(), {
#         "kernel": ["linear", "poly", "rbf", "sigmoid", "precomputed"],
#         "degree": range(1, 6),
#         "gamma": [i / 10.0 for i in range(1, 51)],
#         "coef0": [i / 2.0 for i in range(1, 6)],
#         "shrinking": [True, False],
#         "decision_function_shape": ["ovo", "ovr"],
#         }),
#     "Gaussian Process": (GaussianProcessClassifier(), {
#         "kernel": [
#             WhiteKernel(),
#             ConstantKernel(),
#             RBF(),
#             Matern(),
#             RationalQuadratic(),
#             ExpSineSquared(),
#             DotProduct(),
#             ],
#         "n_restarts_optimizer": range(1, 4),
#         "warm_start": [True, False],
#         "multi_class": ["one_vs_rest", "one_vs_one"],
#         }),
    "Decision Tree": (DecisionTreeClassifier(), {
        "criterion": ["gini", "entropy"],
        "splitter": ["best", "random"],
        "max_depth": [
            1,
            2,
            3,
            4,
            5,
            None
            ],
        "min_samples_split": [i / 20.0 for i in range(1, 21)],
        "min_samples_leaf": [i / 20.0 for i in range(1, 11)],
        "min_weight_fraction_leaf": [i / 10.0 for i in range(6)],
        "max_features": ["sqrt", "log2", None],
        "min_impurity_decrease": [i / 5.0 for i in range(1, 6)],
        "presort": [True, False],
        }),
    "Random Forest": (RandomForestClassifier(), {
        "criterion": ["gini", "entropy"],
        "max_depth": [
            1,
            2,
            3,
            4,
            5,
            None
            ],
        "min_samples_split": [i / 20.0 for i in range(1, 21)],
        "min_samples_leaf": [i / 20.0 for i in range(1, 11)],
        "min_weight_fraction_leaf": [i / 10.0 for i in range(6)],
        "max_features": ["sqrt", "log2", None],
        "min_impurity_decrease": [i / 5.0 for i in range(1, 6)],
        "bootstrap": [True, False],
        }),
    "Neural Net": (MLPClassifier(), {
        "activation": ["identity", "logistic", "tanh", "relu"],
        "solver": ["lbfgs", "sgd", "adam"],
        "alpha": [1.0 / 10 ** i for i in range(3, 9)],
        "learning_rate": ["constant", "invscaling", "adaptive"],
        "shuffle": [True, False],
        "momentum": [i / 5.0 for i in range(6)],
        "validation_fraction": [i / 10.0 for i in range(6)],
        "beta_1": [i / 10.0 for i in range(6, 10)],
        "beta_2": [1 - 1 / 10.0 ** i for i in range(1, 6)],
        }),
    "Naive Bayes": (GaussianNB(), {"var_smoothing": [1 / 10.0 ** i
                    for i in range(3, 20)]}),
    "QDA": (QuadraticDiscriminantAnalysis(), {"reg_param": [i / 10.0
            for i in range(6, 10)]}),
    }

[2018-10-14 22:14:57.927459] Setting up classifier parameter options


Iterate through the classifiers and perform a randomized search on the given parameter ranges storing the best validation and verification scores as well as the average time elapsed.

In [7]:
print "[%s] Starting meta analysis of classifiers" % datetime.datetime.now()

# Number of parameter combinations to evaluate for each classifier
search_iteration_count = 1000

# Maximum number of attempts for randomized search of each classifier
max_retry_count = 10

# Suppress any warnings from classifiers due to random parameter combinations
warnings.filterwarnings("ignore")


# Wrapping call to RandomizedSearchCV prevents invalid parameter combinations
# from stopping overall progress through classifiers
def perform_randomized_search(param_dist, search_iteration_count,
                              retry_count):
    try:
        random_search = RandomizedSearchCV(clf,
                param_distributions=param_dist,
                n_iter=search_iteration_count)
        random_search.fit(x_train_validate, y_train_validate)
        return random_search
    except:
        retry_count += 1
        if retry_count < max_retry_count:
            return perform_randomized_search(param_dist,
                    search_iteration_count, retry_count)
        else:
            return None


# Create lists to store best results for formatting later
best_names = list()
best_validation_scores = list()
best_verification_scores = list()
best_elapsed_times = list()

# Train classifiers and collect results
for (k, v) in clf_opts.iteritems():
    start = datetime.datetime.now()
    print "[%s] Performing randomized search on %s classifier" \
        % (start, k)
    clf = v[0]
    param_dist = v[1]
    random_search = perform_randomized_search(param_dist,
            search_iteration_count, 0)
    elapsed = (datetime.datetime.now() - start).total_seconds() \
        / search_iteration_count
    if random_search:
        clf.set_params(**random_search.best_params_)
        clf.fit(x_train_validate, y_train_validate)
        best_names.append(k)
        best_validation_scores.append(random_search.best_score_)
        best_verification_scores.append(clf.score(x_verify, y_verify))
        best_elapsed_times.append(elapsed)
    else:
        print "[%s] Maximum number of randomized search attempts for %s met." \
            % (datetime.datetime.now(), k)

[2018-10-14 22:14:57.968548] Starting meta analysis of classifiers
[2018-10-14 22:14:57.969594] Performing randomized search on QDA classifier
[2018-10-14 22:14:58.058093] Performing randomized search on Decision Tree classifier
[2018-10-14 22:15:07.247340] Performing randomized search on Naive Bayes classifier
[2018-10-14 22:15:07.406846] Performing randomized search on Neural Net classifier
[2018-10-14 22:51:33.786657] Performing randomized search on Random Forest classifier
[2018-10-14 22:53:14.387423] Performing randomized search on Nearest Neighbors classifier


Display the classifier results sorted in descending order by verification score and ascending average time elapsed.

In [8]:
print "[%s] Formatting results of randomized parameter search\n" % \
    datetime.datetime.now()

# Construct dataframe to sort and display results
formatted_results = pandas.DataFrame.from_dict({
    "Classifier": best_names,
    "Validation Score": best_validation_scores,
    "Verification Score": best_verification_scores,
    "Average Time (Seconds)": best_elapsed_times,
    })

# Move classifier column to index
formatted_results.index = formatted_results.pop("Classifier")
formatted_results.index.name = "Classifier"

# Arrange columns in order for display table
formatted_results = formatted_results[["Validation Score",
        "Verification Score", "Average Time (Seconds)"]]

# Sort classifiers by descending verification score and ascending average time
formatted_results.sort_values(by=["Verification Score",
                              "Average Time (Seconds)"],
                              ascending=[False, True], inplace=True)

pandas.set_option('display.width', 120)

print formatted_results

[2018-10-14 22:55:57.258555] Formatting results of randomized parameter search

                   Validation Score  Verification Score  Average Time (Seconds)
Classifier                                                                     
Decision Tree              0.992087            0.987516                0.009187
Random Forest              0.992087            0.987516                0.100572
Nearest Neighbors          0.992503            0.987516                0.162836
Neural Net                 0.992087            0.987516                2.185270
Naive Bayes                0.987922            0.977528                0.000157
QDA                        0.958351            0.955056                0.000085
